In [8]:
theta1 = [0.31, 0.18, 0.65, 0.25, 5]
theta_n1 = [val * 1.4 for val in theta1]
theta_min1 = [val * 0.2 for val in theta1]
theta_max1 = [val * 1.8 for val in theta1]
theta_maxs1 = [hi / nom for hi, nom in zip(theta_max1, theta_n1)]
theta_mins1 = [lo / nom for lo, nom in zip(theta_min1, theta_n1)]

In [9]:
system = {
    'tvi': {  # Time-variant input variables (models input: tvi), each key is a symbol nad key in tvi as well
        'u1': {  # Temperature (K)
            'stps': 5,  # Number of switching times in CVPs (vector parametrisation resolution in time dimension):
            # Must be a positive integer > 1. swps-1 is the number of steps
            'const': 'rel',  # Constraint type: relative state of signal levels in CVPs
            # 'rel' (relative) ensures relaxation, 'dec' (decreasing) ensures decreasing signal levels, 'inc' (increasing) ensures increasing signal levels
            'max': 0.2,  # Maximum allowable signal level, des_opt space upper bound
            'min': 0.05,  # Minimum allowable signal level, des_opt space lower bound
            'cvp': 'CPF',  # Design CVP method (CPF - constant profile, LPF - linear profile)
            'offl': 0.01,  # minimum allowed perturbation of signal (ratio)
            'offt': 0.5  # minimum allowed perturbation of time (ratio)
        },
        'u2': {  # Temperature (K)
            'stps': 5,  # Number of switching times in CVPs (vector parametrisation resolution in time dimension):
            # Must be a positive integer > 1. swps-1 is the number of steps
            'const': 'dec',  # Constraint type: relative state of signal levels in CVPs
            # 'rel' (relative) ensures relaxation, 'dec' (decreasing) ensures decreasing signal levels, 'inc' (increasing) ensures increasing signal levels
            'max': 35,  # Maximum allowable signal level, des_opt space upper bound
            'min': 5,  # Minimum allowable signal level, des_opt space lower bound
            'cvp': 'CPF',  # Design CVP method (CPF - constant profile, LPF - linear profile)
            'offl': 5,  # minimum allowed perturbation of signal (ratio)
            'offt': 0.5  # minimum allowed perturbation of time (ratio)
        },
    },
    'tvo': {  # Time-variant output variables (responses, measured or unmeasured)
        'y1': {  # response variable, here carbonation efficiency
            'init': 0,  # Initial value for the response variable, it can be a value, or 'variable' for case it is a des_opt decision (time-invariant input variable)
            'meas': True,  # Flag indicating if this variable is directly measurable, if False, it is a virtual output
            'sp': 6,  # the amound of samples per each round (run)
            'unc': 0.05,  # amount of noise (standard deviation) in the measurement, in case of insilico, this is used for simulating a normal distribution of noise to measurement (only measurement)
            'offt': 0.5,  # minimum allowed perturbation of sampling times (ratio)
            'samp_s': 1,  # Matching criterion for models prediction and data alignment
            'samp_f': [0, 10],  # fixed sampling times
        },
        'y2': {  # response variable, here carbonation efficiency
            'init': 0,  # Initial value for the response variable, it can be a value, or 'variable' for case it is a des_opt decision (time-invariant input variable)
            'meas': True,  # Flag indicating if this variable is directly measurable, if False, it is a virtual output
            'sp': 6,  # the amound of samples per each round (run)
            'unc': 0.05,  # amount of noise (standard deviation) in the measurement, in case of insilico, this is used for simulating a normal distribution of noise to measurement (only measurement)
            'offt': 0.5,  # minimum allowed perturbation of sampling times (ratio)
            'samp_s': 1,  # Matching criterion for models prediction and data alignment
            'samp_f': [0, 10],  # fixed sampling times
        },
    },
    'tii': {  # Time-invariant input variables (tii)
        'y10': {  # 1st symbolic time-invariant control, Density of solid reactant (kg/m³)
            'max': 10,  # Maximum allowable signal level, des_opt space upper bound
            'min': 1  # Minimum allowable signal level, des_opt space upper bound
        },
        'y20': {  # 1st symbolic time-invariant control, Density of solid reactant (kg/m³)
            'max': 10,  # Maximum allowable signal level, des_opt space upper bound
            'min': 1  # Minimum allowable signal level, des_opt space upper bound
        },
    },
    'tio': {  # Time-invariant output variables (empty here, could hold steady state responses that hold no dependency)
    },
    't_s': [0, 10],  # Time span  (600 s to 10,800 s), duration of numerical perturbations (the rest is precluded from des_opt)
    't_r': 0.02,  # Time resolution (10 s), minimum time steps for the simulation/des_opt/controls
    't_d': 0.5
}

In [10]:
models = { # Settings related to the rival models and their parameters
    'can_m': ['M'],  # Active solvers (rival models) to be used in the experiment
    'krt': {'M': 'pys'},  # Kernel type for each model, 'pys' for python standalone scripts, 'pym' for middoe.krnl_models, 'gpr' for gPAS models
    # type of the model interface, 'pym' for middoe.krnl_models, 'gpr' for gPAS models, function name for globally defined functions, 'pys' for python standalone scripts
    'creds': {'M': '@@TTmnoa698'},
    # credentials for gPAS models, if not needed, leave empty
    'src': {'M': 'C:/Users/Tadmin/PycharmProjects/middoe/tests/case study poster/model.py'},
    # for now for gPAS readable files, or python standalone scripts

    'theta': { # Theta parameters for each models
        'M': theta_n1
    },
    't_u': { # Maximum bounds for theta parameters (based on normalized to'f20': theta20mins, 1)
        'M': theta_maxs1
    },
    't_l': { # Minimum bounds for theta parameters (based on normalized to 1)
        'M': theta_mins1
    }
}

In [11]:
insilicos = { # Settings for the insilico data generation
    'tr_m': 'M', # selected true models (with nominal values)
    'theta': theta1,
    'errt': 'abs',  # error type, 'rel' for relative error, 'abs' for absolute error
    'prels': { # classic des_opt settings, sheet name is the round run name, each sheet contains the data for the round, iso space.
        '1': {'u1': 0.125, 'u2':20, 'y10': 5.5, 'y20': 5.5},
        '2': {'u1': 0.2, 'u2':35, 'y10': 10, 'y20': 10},
        '3': {'u1': 0.05, 'u2':5, 'y10': 1, 'y20': 1},
    }
}

In [12]:
from middoe.krnl_expera import expera
expera(system, models, insilicos, design_decisions={}, expr=1)


[✓] In-silico data saved to: C:\Users\Tadmin\PycharmProjects\middoe\tests\case study poster\indata.xlsx
[INFO] Model used         : M
[INFO] Design case        : classic/preliminary
[INFO] Responses simulated:
   - y1         | measurable      | std.dev = 0.05
   - y2         | measurable      | std.dev = 0.05


(WindowsPath('C:/Users/Tadmin/PycharmProjects/middoe/tests/case study poster/indata.xlsx'),
      MES_X:y1  MES_Y:y1  MES_X:y2  MES_Y:y2  X:all  y10  y20     u1    u2  \
 0         0.0  5.389935       0.0  5.410285   0.00  5.5  5.5  0.125  20.0   
 1         2.0  3.355977       2.0  1.867120   0.02  5.5  5.5  0.125  20.0   
 2         4.0  1.844734       4.0  0.016674   0.04  5.5  5.5  0.125  20.0   
 3         6.0  0.915667       6.0  0.013553   0.06  5.5  5.5  0.125  20.0   
 4         8.0  0.510391       8.0  0.031224   0.08  5.5  5.5  0.125  20.0   
 ..        ...       ...       ...       ...    ...  ...  ...    ...   ...   
 496       NaN       NaN       NaN       NaN   9.92  5.5  5.5  0.125  20.0   
 497       NaN       NaN       NaN       NaN   9.94  5.5  5.5  0.125  20.0   
 498       NaN       NaN       NaN       NaN   9.96  5.5  5.5  0.125  20.0   
 499       NaN       NaN       NaN       NaN   9.98  5.5  5.5  0.125  20.0   
 500       NaN       NaN       NaN       NaN  10.0

In [7]:
expera(system, models, insilicos, design_decisions={}, expr=2)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Tadmin\\PycharmProjects\\middoe\\tests\\case study poster\\indata.xlsx'

In [63]:
expera(system, models, insilicos, design_decisions={}, expr=3)


[✓] In-silico data saved to: C:\Users\Tadmin\PycharmProjects\middoe\tests\case study poster\indata.xlsx
[INFO] Model used         : M
[INFO] Design case        : classic/preliminary
[INFO] Responses simulated:
   - y1         | measurable      | std.dev = 0.0001
   - y2         | measurable      | std.dev = 0.0001


(WindowsPath('C:/Users/Tadmin/PycharmProjects/middoe/tests/case study poster/indata.xlsx'),
      MES_X:y1  MES_Y:y1  MES_X:y2  MES_Y:y2  X:all  y10  y20    u1   u2  \
 0         0.0  1.000139       0.0  1.000015   0.00  1.0  1.0  0.05  5.0   
 1         2.0  0.844691       2.0  0.876264   0.02  1.0  1.0  0.05  5.0   
 2         4.0  0.713741       4.0  0.768302   0.04  1.0  1.0  0.05  5.0   
 3         6.0  0.601680       6.0  0.675583   0.06  1.0  1.0  0.05  5.0   
 4         8.0  0.505100       8.0  0.596608   0.08  1.0  1.0  0.05  5.0   
 ..        ...       ...       ...       ...    ...  ...  ...   ...  ...   
 496       NaN       NaN       NaN       NaN   9.92  1.0  1.0  0.05  5.0   
 497       NaN       NaN       NaN       NaN   9.94  1.0  1.0  0.05  5.0   
 498       NaN       NaN       NaN       NaN   9.96  1.0  1.0  0.05  5.0   
 499       NaN       NaN       NaN       NaN   9.98  1.0  1.0  0.05  5.0   
 500       NaN       NaN       NaN       NaN  10.00  1.0  1.0  0.05  5.0

In [68]:
iden_opt = { # Settings for the parameter estimation process
    'meth': 'G',  # optimisation method, 'G': Global Differential Evolution, 'Ls': Local SLSQP, 'Ln': Local Nelder-Mead
    'init': None,   # use 'rand' to have random starting point and use None to start from theta_parameters nominal values (to be avoided in insilico studies)
    'eps': 1e-3,  # perturbation size of parameters in SA FDM method (in a normalized to 1 space)
    #usually 1e-3, or None to perform a mesh independency test, and auto adjustment
    'ob': 'WLS',  #loss function, 'LS': least squares, 'MLE': maximum likelihood, 'Chi': chi-square, 'WLS': weighted least squares
    'c_plt': True, # plot the confidence volumes
    'f_plt': True, # plot the fitting results
    'plt_s': True, # show plots while saving
    'log': True # log the results
}

In [65]:
from middoe.log_utils import  read_excel
data = read_excel('indata')

[INFO] Reading from indata.xlsx


In [ ]:
from middoe.iden_parmest import parmest
resultpr = parmest(system, models, iden_opt, data)

In [67]:
from middoe.iden_uncert import uncert
uncert_results = uncert(data, resultpr, system, models, iden_opt)
resultun = uncert_results['results']
theta_parameters = uncert_results['theta_parameters']
solver_parameters = uncert_results['solver_parameters']
scaled_params = uncert_results['scaled_params']
obs = uncert_results['obs']

Estimated parameters of M: [0.434, 0.252, 0.9099999999999999, 0.35, 7.0]
True parameters of M: [0.434, 0.252, 0.9099999999999999, 0.35, 7.0]
LS objective function value for M: 0.9806510304505232
T-values of model M: [0.56735425 0.17374414 2.03244744 2.94952284 0.44356642]
P-value of model:M is 100.0 for model discrimination
R2 values for responses in model M:
  y1: 0.9871
  y2: 0.9858


In [ ]:
from middoe.log_utils import  save_rounds
round_data={}
round = 1
save_rounds(round, resultun, theta_parameters, 'preliminary', round_data, models, scaled_params,iden_opt,solver_parameters, obs, data, system)